In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Add
from tensorflow.keras.models import Model
import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


input_shape = (48, 48, 1)
dataset_path = "/kaggle/input/fer2013-cleaned-dataset/images1"
batch_size = 64
num_train_samples = 28044
num_test_samples = 7177

train_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

train_data = train_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'train'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_data = test_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'test'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)



from keras.models import Sequential
import numpy as np
import tensorflow as tf
np.random.seed(42)
tf.random.set_seed(42)




/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Found 28044 images belonging to 7 classes.
Found 7177 images belonging to 7 classes.


In [2]:
def residual_block(x, filters, kernel_size=(3, 3), stride=(1, 1), padding='same'):
    shortcut = x
    
    # First convolution layer
    x = Conv2D(filters, kernel_size, strides=stride, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Second convolution layer
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    
    # Add the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    
    return x


def build_vgg_with_resnet(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    
    # VGG-like convolutional layers with residual blocks
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_layer)
    x = residual_block(x, 64)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = residual_block(x, 128)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = residual_block(x, 256)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = residual_block(x, 512)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Flatten and add fully connected layers
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    
    # Output layer
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model




In [3]:
num_classes = 7  
model = build_vgg_with_resnet(input_shape, num_classes)

In [4]:
optimizer = tf.keras.optimizers.Adam(amsgrad=True,learning_rate=0.001)  # Using Adam with AMSGrad
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(
   train_data,
     steps_per_epoch=num_train_samples // batch_size,
     epochs=30,
    batch_size=batch_size,
    validation_data=test_data
)

Epoch 1/30
438/438 [==============================] - 107s 219ms/step - loss: 2.0035 - accuracy: 0.2520 - val_loss: 1.7669 - val_accuracy: 0.2922
Epoch 2/30
438/438 [==============================] - 34s 79ms/step - loss: 1.5624 - accuracy: 0.3766 - val_loss: 1.5435 - val_accuracy: 0.3837
Epoch 3/30
438/438 [==============================] - 37s 84ms/step - loss: 1.3682 - accuracy: 0.4637 - val_loss: 1.4986 - val_accuracy: 0.4039
Epoch 4/30
438/438 [==============================] - 39s 89ms/step - loss: 1.2681 - accuracy: 0.5044 - val_loss: 1.4027 - val_accuracy: 0.4622
Epoch 5/30
438/438 [==============================] - 35s 79ms/step - loss: 1.1905 - accuracy: 0.5406 - val_loss: 1.2703 - val_accuracy: 0.5044
Epoch 6/30
438/438 [==============================] - 32s 74ms/step - loss: 1.1338 - accuracy: 0.5633 - val_loss: 1.2081 - val_accuracy: 0.5343
Epoch 7/30
438/438 [==============================] - 33s 75ms/step - loss: 1.0784 - accuracy: 0.5909 - val_loss: 1.2643 - val_accurac

In [5]:
test_loss, test_accuracy = model.evaluate(
    test_data,
    steps=num_test_samples // batch_size
)

112/112 [==============================] - 6s 55ms/step - loss: 2.3863 - accuracy: 0.5917


In [6]:
from sklearn.metrics import classification_report, roc_auc_score

# Assuming you have trained your model and evaluated it
# test_data, num_test_samples, batch_size, model

# Make predictions on test data
predictions = model.predict(test_data, steps=num_test_samples // batch_size)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_data.classes[:len(predicted_classes)]  # Extract true classes from the test_data

# Calculate AUC score
auc = roc_auc_score(test_data.labels[:len(predicted_classes)], predictions, multi_class='ovr') 

# print("AUC Score:", auc)


112/112 [==============================] - 7s 61ms/step


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_true = test_data.classes  # Assuming you're using flow_from_directory
predictions = model.predict(test_data)

# Convert predictions to class labels
y_pred = np.argmax(predictions, axis=1)

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# If binary classification, compute AUC  # AUC is not applicable for multi-class
print("AUC Score:", auc)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

113/113 [==============================] - 6s 53ms/step
AUC Score: 0.8658179809575918
Accuracy: 0.5913334262226557
Precision: 0.5900274645421822
Recall: 0.5913334262226557
F1 Score: 0.5803133534784726
